# Arguments

In [1]:
batch_size = 32
dataset_path_plantvillage = '../datasets/plantvillage'
dataset_path_self = '../datasets/self'
dataset_path_validation = '../datasets/validation'

dataset_paths = [dataset_path_plantvillage, dataset_path_self, dataset_path_validation]

warmup_model_path = 'model-warmup.h5'
model_save_path = 'model-final.h5'
checkpoint_path = 'checkpoints-finetuning.hdf5'

input_width = 224
input_height = 224
input_depth = 3

# --------------------------------------------------
num_of_epochs = 100
start_epoch = 0
# --------------------------------------------------


# Select training classes

In [2]:
class_paths_training = ['../datasets/self/___Early_blight', '../datasets/self/___Appids', '../datasets/self/___Leaf_miner', '../datasets/self/___Curly_top_virus']

# Load dataset

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.resize_image_preprocessor import resizeImagePreprocessor
from utils.preprocessors.img_to_array_preprocessor import ImgToArrayPreprocessor
from utils.io.dataset_loader import DatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

def load_datasets(path_list):
    # Load image paths
    image_paths = []
    print("[INFO] loading images...")
    for path in path_list:
        image_paths.extend(list(paths.list_images(path)))
        
    # Get unique classnames

    class_names = [pt.split(os.path.sep)[-2] for pt in image_paths]
    class_names = [str(x) for x in np.unique(class_names)]

    # Initial image preprocessing
    aap = resizeImagePreprocessor(input_width, input_height)
    iap= ImgToArrayPreprocessor()

    #Load image data and perform image data preprocessing
    dl = DatasetLoader(preprocessors=[aap,iap])
    (data,labels)  = dl.load(image_paths,verbose=500)
    data = data.astype("float") / 255.0


    # train test split
    (train_x,test_x,train_y,test_y) = train_test_split(data,labels,test_size=0.25,random_state=42)

    # convert the labels from integers to vectors
    train_y = LabelBinarizer().fit_transform(train_y)
    test_y = LabelBinarizer().fit_transform(test_y)
    
    return (train_x,test_x,train_y,test_y, class_names)

(train_x,test_x,train_y,test_y, class_names) = load_datasets(class_paths_training)



Using TensorFlow backend.


[INFO] loading images...
[INFO]: Processed 500/4849
[INFO]: Processed 1000/4849
[INFO]: Processed 1500/4849
[INFO]: Processed 2000/4849
[INFO]: Processed 2500/4849
[INFO]: Processed 3000/4849
[INFO]: Processed 3500/4849
[INFO]: Processed 4000/4849
[INFO]: Processed 4500/4849


# Load model from disk

In [4]:
# import tensorflow as tf
# print(tf.__version__)


# model = tf.keras.models.load_model(
#     warmup_model_path,
#     custom_objects=None,
#     compile=False
# )

from keras.models import load_model

model = load_model(warmup_model_path)

# Print index of layers

In [5]:
# iterate for all layers in the network and print its' index value
for (i,layer) in enumerate(model.layers):
    print("[INFO] {:5}\t{:30}{}".format(i, layer.name, layer.__class__.__name__))

[INFO]     0	input_2                       InputLayer
[INFO]     1	conv1_pad                     ZeroPadding2D
[INFO]     2	conv1                         Conv2D
[INFO]     3	bn_conv1                      BatchNormalization
[INFO]     4	activation_50                 Activation
[INFO]     5	pool1_pad                     ZeroPadding2D
[INFO]     6	max_pooling2d_2               MaxPooling2D
[INFO]     7	res2a_branch2a                Conv2D
[INFO]     8	bn2a_branch2a                 BatchNormalization
[INFO]     9	activation_51                 Activation
[INFO]    10	res2a_branch2b                Conv2D
[INFO]    11	bn2a_branch2b                 BatchNormalization
[INFO]    12	activation_52                 Activation
[INFO]    13	res2a_branch2c                Conv2D
[INFO]    14	res2a_branch1                 Conv2D
[INFO]    15	bn2a_branch2c                 BatchNormalization
[INFO]    16	bn2a_branch1                  BatchNormalization
[INFO]    17	add_17                        Add
[INFO] 

# Unfreeze final CONV layers

In [9]:
for layer in model.layers[:155]:
   layer.trainable = False
for layer in model.layers[155:]:
   layer.trainable = True
print('unfrozen')

unfrozen


# Compile model

In [10]:
from keras.optimizers import SGD

# Build the model from the new
print("[INFO] re-compiling model ...")
opt = SGD(lr=0.0001, momentum=0.09)
# Fine-tuning with a small learning rate
model.compile(loss = 'categorical_crossentropy',optimizer = opt,
              metrics=['accuracy'])


[INFO] re-compiling model ...


# Checkpoints

In [11]:
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', 
save_best_only=True, verbose=1)

callbacks = [checkpoint]



In [12]:
#load checkpoints if existing

import os

if(os.path.exists(checkpoint_path)):
    model.load_weights(checkpoint_path)

# Training Monitor

In [13]:
from utils.callbacks.monitor import Monitor
import pathlib
import json
import os

fig_path = "plot"
json_path = "values.json"
values_path = 'values.json'

callbacks.append(Monitor(fig_path, json_path, start_epoch))

# Fit model

In [14]:
num_of_epochs = 100

H = model.fit_generator(
    aug.flow(train_x,train_y, batch_size = 32),
             validation_data = (test_x,test_y),
             epochs=num_of_epochs,
             steps_per_epoch = len(train_x) //32,
             verbose = 1,
             callbacks=callbacks)

Epoch 1/100
113/113 [==============================] - 70s 621ms/step - loss: 0.0427 - accuracy: 0.9878 - val_loss: 2.2431 - val_accuracy: 0.1171

Epoch 00001: val_loss improved from inf to 2.24312, saving model to checkpoints-finetuning.hdf5
Epoch 2/100
113/113 [==============================] - 62s 551ms/step - loss: 0.0461 - accuracy: 0.9836 - val_loss: 2.2269 - val_accuracy: 0.1171

Epoch 00002: val_loss improved from 2.24312 to 2.22691, saving model to checkpoints-finetuning.hdf5
Epoch 3/100
113/113 [==============================] - 61s 544ms/step - loss: 0.0485 - accuracy: 0.9850 - val_loss: 2.2688 - val_accuracy: 0.1171

Epoch 00003: val_loss did not improve from 2.22691
Epoch 4/100
113/113 [==============================] - 61s 538ms/step - loss: 0.0467 - accuracy: 0.9825 - val_loss: 2.2758 - val_accuracy: 0.1171

Epoch 00004: val_loss did not improve from 2.22691
Epoch 5/100
113/113 [==============================] - 62s 551ms/step - loss: 0.0458 - accuracy: 0.9825 - val_loss

# Save model

In [15]:
model.save(model_save_path)

# Evaluate with 20%

In [16]:
from sklearn.metrics import classification_report

print("[INFO] evaluating after initialization...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))

[INFO] evaluating after initialization...
                    precision    recall  f1-score   support

         ___Appids     0.0000    0.0000    0.0000       242
___Curly_top_virus     0.1171    1.0000    0.2096       142
   ___Early_blight     0.0000    0.0000    0.0000       384
     ___Leaf_miner     0.0000    0.0000    0.0000       445

       avg / total     0.0137    0.1171    0.0245      1213



/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Evaluate with validation images

In [17]:
class_paths_validation = ['../datasets/validation/___Early_blight', '../datasets/validation/___Appids', '../datasets/validation/___Leaf_miner', '../datasets/self/___Curly_top_virus']

In [18]:
(train_x,test_x,train_y,test_y, class_names) = load_datasets(class_paths_validation)

from sklearn.metrics import classification_report

print("[INFO] evaluating with validation set...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))

[INFO] loading images...
[INFO]: Processed 500/614
[INFO] evaluating with validation set...
                    precision    recall  f1-score   support

         ___Appids     0.0000    0.0000    0.0000         9
___Curly_top_virus     0.7662    1.0000    0.8676       118
   ___Early_blight     0.0000    0.0000    0.0000        12
     ___Leaf_miner     0.0000    0.0000    0.0000        15

       avg / total     0.5871    0.7662    0.6648       154

